In [2]:
import pandas as pd
from sklearn.cluster import KMeans

In [5]:
df=pd.read_csv("./datasets/output.csv")
df

,genes,GSM742944,GSM2326089,GSM1807974,GSM1807990,GSM2055788,GSM2142335,GSM1807979,GSM1695909,GSM1554468,...,GSM2653842,GSM2653843,GSM2653844,GSM2653845,GSM2653846,GSM2653847,GSM2653849,GSM2653850,GSM2653851,GSM2653853
0,A1BG,0,92,11663,2089,18808,183,5123,79640,1595,...,30,22,17,49,394,12,8,6,10,8
1,A1CF,0,17,5738,490,14035,5,7127,10168,241,...,9,4,15,19,105,5,8,4,2,24
2,A2M,6362,304,39269,8299,20260,97,27210,63297,5817,...,8,6,2,0,374,0,4,1,3,1
3,A2ML1,0,22,11,2,39,8,4,9,5,...,4,5,1,4,1,6,3,3,3,1
4,A2MP1,0,3,167,7,44,15,16,61,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35233,BP-2171C21.4,0,0,0,0,0,0,0,1,0,...,15,17,13,8,2,0,0,0,0,0
35234,BP-2171C21.5,0,0,0,0,8,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35235,BP-2171C21.6,0,4,1,0,1,6,1,1,0,...,0,0,0,0,2,1,0,0,0,1
35236,BP-2189O9.2,0,25,6,0,48,110,2,1,0,...,0,0,0,1,1,0,0,0,0,0
